# Install the library to scrape the wiki website

In [1]:
!pip install beautifulsoup4

# Install the parser recommended

In [2]:
!pip install lxml

# Install request to get info from www

In [3]:
!pip install request

# Importing function and libraries

In [4]:
import requests  as myrq # library for requests
import lxml.html as mylh # library for handle html from wiki
from bs4 import BeautifulSoup #library for scraping web
import numpy as mynp # library to handle data in a vector
import pandas as mypd # library for data analsysis

In [7]:
Url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #to get data

# Get the whole html information from wiki

In [8]:
mysource = myrq.get(Url).text

# Html unreadable

In [9]:
Toronto_Info = BeautifulSoup(mysource, 'lxml')
#print (Toronto_Info)

# Make this html readable

In [10]:
#print(Toronto_Info.prettify())

# Get the table info from "<!--table class="wikitable sortable"--!>"

In [11]:
htmltable = Toronto_Info.find('table',{'class':'wikitable sortable'})

In [12]:
#print (htmltable.prettify())

# Get all rows with postcodes

In [13]:
htmltablerows = htmltable.find_all('tr')
data = []
for row in htmltablerows:
    td=[]
    for t in row.find_all('td'):
        td.append(t.text.strip())
    data.append(td)

# Insert rows and columns inside a Pandas DataFrame

In [14]:
mydf = mypd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])

In [15]:
mydf

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned


# Cleand Data: Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [17]:
mydf.drop(mydf[mydf.Borough == 'Not assigned'].index, inplace=True) # Not assigned values in Borough
mydf = mydf[~mydf['Borough'].isnull()]  # null values in Borough
mydf.drop(mydf[mydf.Borough == 'None'].index, inplace=True) # None values in Borough
mydf.reset_index(drop=True, inplace=True)

## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [18]:
mydf['Neighborhood'].replace('Not assigned',mydf['Borough'],inplace=True)

## More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [19]:
mydf = mydf.groupby(['PostalCode','Borough'],sort=False)['Neighborhood'].apply(', '.join).reset_index()
mydf.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


## Checking null values


In [20]:
print(mydf.PostalCode.isnull().count())
print(mydf.Borough.isnull().count())
print(mydf.Neighborhood.isnull().count())

103
103
103


## Dataframe shape

In [21]:
mydf.shape

(103, 3)

# Longitude coordinates of each neighborhood to use Foursquare location data.

In [22]:
Urlgeo ="http://cocl.us/Geospatial_data"
myGeoDf = mypd.read_csv(Urlgeo)

In [23]:
myGeoDf.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Combining data with the geo data

In [24]:
myGeoDf.columns = ['PostalCode', 'Latitude', 'Longitude']

In [25]:
mydfgeodata = mypd.merge(myGeoDf, mydf, on='PostalCode')
mydfgeodata.head(5)

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


# Sorting the dataframe with new columns

In [26]:
mydfgeodata = mydfgeodata[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
mydfgeodata.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [27]:
mydfgeodata[mydfgeodata.PostalCode == 'M5G'] #Taking postal code M5G as an example as mentioned in the script

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [28]:
mydfgeodata.shape

(103, 5)

# Instal all the libraries for Plotting and clustering

In [29]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [30]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


# Create map of Toronto using latitude and longitude values, and renderit with folium

In [31]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, borough, neighborhood in zip(mydfgeodata['Latitude'], mydfgeodata['Longitude'], mydfgeodata['Borough'], mydfgeodata['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [32]:
# @hidden_cell
CLIENT_ID = 'SJKE1OH0FGMJQJUI5PWEMJT2C1MKK0IG2HHXMHRUCGGEU2LM' # your Foursquare ID
CLIENT_SECRET = 'IIID2UGPF1QVHND5JQFIUMDUXGDRURZX21QAYRGS4Z3FLNSV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100 # limit of number of venues returned 
RADIUS = 500 # define radius

Your credentails:


# Ramdon neighborhood from data set

In [33]:
mydfgeodata.loc[4,'Neighborhood']

'Cedarbrae'

In [34]:
Seletedneigh_latitude = mydfgeodata.loc[4, 'Latitude'] 
Seletedneigh_longitude = mydfgeodata.loc[4, 'Longitude']
Seletedneigh_Name = mydfgeodata.loc[4,'Neighborhood']
print('Latitude and longitude values of {} are {}, {}.'.format(Seletedneigh_Name,Seletedneigh_latitude,Seletedneigh_longitude))

Latitude and longitude values of Cedarbrae are 43.773136, -79.23947609999999.


# Url to foursquare

In [35]:
Url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, Seletedneigh_latitude, Seletedneigh_longitude, RADIUS, LIMIT)
Myresults = myrq.get(Url).json()

In [36]:
Myresults

{'meta': {'code': 200, 'requestId': '5db20dd4446ea60039c4459e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Woburn',
  'headerFullLocation': 'Woburn, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 43.7776360045, 'lng': -79.2332557734104},
   'sw': {'lat': 43.7686359955, 'lng': -79.24569642658957}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e261f261f6eb1ae13930699',
       'name': "Drupati's Roti & Doubles",
       'location': {'address': '1085 Bellamy Rd N',
        'crossStreet': 'Bellamy & Ellesmere',
        'lat': 43.775222138791534,
        'lng': -79.24167761001029,
        'labeledLatLngs': [{'lab

# function that extracts the category of the venue

In [37]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# Now we are ready to clean the json and structure it into a pandas dataframe.

In [38]:
venues = Myresults['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

nearby_venues.head()

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Drupati's Roti & Doubles,Caribbean Restaurant,43.775222,-79.241678
1,Federick Restaurant,Hakka Restaurant,43.774697,-79.241142
2,Centennial Recreation Centre,Athletics & Sports,43.774593,-79.236500
3,Thai One On,Thai Restaurant,43.774468,-79.241268
4,TD Canada Trust,Bank,43.774952,-79.241343


In [39]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

7 venues were returned by Foursquare.


#  Neighborhood to Toronto

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = myrq.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = mypd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
mydfgeodataToronto = mydfgeodata

mydfgeodataToronto.drop(mydfgeodataToronto[mydfgeodataToronto.Borough.str.contains("Toronto")==False].index, inplace=True)
mydfgeodataToronto = mydfgeodataToronto.reset_index(drop=True)
mydfgeodataToronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [58]:
Toronto_venues = getNearbyVenues(names=mydfgeodata['Neighborhood'],latitudes=mydfgeodata['Latitude'],longitudes=mydfgeodata['Longitude'])

#Toronto_venues

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [59]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1712, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [61]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",24,24,24,24,24,24
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,86,86,86,86,86,86
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,17,17,17,17,17,17


In [63]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


# Analyze Each Neighborhood

In [64]:
# one hot encoding
Toronto_onehot = mypd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
Toronto_onehot.shape

(1712, 237)

In [67]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.00000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.01
1,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.00000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.052632,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.00,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown, St. James Town",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.011628,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.00000,0.00,0.011628,0.000000,0.000000,0.011628,0.000000,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.010000,0.00000,0.00,0.050000,0.000000,0.040000,0.010000,0.000000,0.00
8,Christie,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.011236,0.00,0.011236,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.00000,0.00,0.000000,0.011236,0.000000,0.000000,0.011236,0.00


In [69]:
Toronto_grouped.shape

(38, 237)

In [70]:
num_top_venues = 10

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2                  Bar  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.03
5                Hotel  0.03
6       Breakfast Spot  0.03
7               Bakery  0.03
8           Restaurant  0.03
9     Asian Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2          Steakhouse  0.04
3  Seafood Restaurant  0.04
4              Bakery  0.04
5      Farmers Market  0.04
6         Cheese Shop  0.04
7                Café  0.04
8            Beer Bar  0.04
9  Italian Restaurant  0.04


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0                  Bakery  0.08
1          Breakfast Spot  0.08
2                    Café  0.08
3             Coffee Shop  0.08
4            Intersection  0.04
5      Italian Restaurant  0.04
6    Caribbean Restaurant  0.04
7      

                     venue  freq
0                      Bar  0.11
1              Coffee Shop  0.06
2         Asian Restaurant  0.05
3                     Café  0.03
4               Restaurant  0.03
5                   Bakery  0.03
6              Men's Store  0.03
7  New American Restaurant  0.03
8              Pizza Place  0.03
9        French Restaurant  0.03


----Moore Park, Summerhill East----
                      venue  freq
0                Playground  0.25
1                      Park  0.25
2              Tennis Court  0.25
3                     Trail  0.25
4          Malay Restaurant  0.00
5                    Market  0.00
6         Martial Arts Dojo  0.00
7  Mediterranean Restaurant  0.00
8             Movie Theater  0.00
9               Men's Store  0.00


----North Toronto West----
                    venue  freq
0     Sporting Goods Shop  0.10
1             Coffee Shop  0.10
2          Clothing Store  0.10
3             Yoga Studio  0.05
4     Rental Car Location  0.05
5   

In [71]:
#First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [76]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in mynp.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = mypd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in mynp.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Thai Restaurant,Steakhouse,Sushi Restaurant,Breakfast Spot,Hotel,Bakery,Asian Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Steakhouse,Farmers Market,Beer Bar,Cheese Shop,Café,Seafood Restaurant,Italian Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Bakery,Gym,Caribbean Restaurant,Stadium,Sandwich Place,Burrito Place,Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Butcher,Smoke Shop,Spa,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Bar,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Boutique


# Cluster Neighborhoods

In [78]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [79]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = mydfgeodataToronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Trail,Pub,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Bakery,Indian Restaurant,Pub,Dessert Shop,Sports Bar
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Sandwich Place,Gym,Park,Board Shop,Brewery,Burger Joint,Burrito Place,Pub,Movie Theater,Sushi Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Brewery,Bank,Bar,Fish Market,Bookstore
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


# visualize the resulting clusters

In [82]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = mynp.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(mynp.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster 1

In [83]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Bakery,Indian Restaurant,Pub,Dessert Shop,Sports Bar
2,East Toronto,0,Sandwich Place,Gym,Park,Board Shop,Brewery,Burger Joint,Burrito Place,Pub,Movie Theater,Sushi Restaurant
3,East Toronto,0,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Brewery,Bank,Bar,Fish Market,Bookstore
5,Central Toronto,0,Gym,Park,Clothing Store,Sandwich Place,Food & Drink Shop,Hotel,Breakfast Spot,Diner,Dessert Shop,Dim Sum Restaurant
6,Central Toronto,0,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Furniture / Home Store,Diner,Dessert Shop,Mexican Restaurant,Park,Chinese Restaurant
7,Central Toronto,0,Sandwich Place,Pizza Place,Dessert Shop,Coffee Shop,Gym,Toy / Game Store,Café,Italian Restaurant,Sushi Restaurant,Flower Shop
9,Central Toronto,0,Pub,Coffee Shop,Light Rail Station,Sushi Restaurant,Supermarket,Bagel Shop,Fried Chicken Joint,Sports Bar,American Restaurant,Restaurant
11,Downtown Toronto,0,Pizza Place,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Café,Pub,Park,Diner,Market
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Gym,Pub,Men's Store,Mediterranean Restaurant
13,Downtown Toronto,0,Coffee Shop,Park,Bakery,Café,Mexican Restaurant,Breakfast Spot,Pub,Theater,Cosmetics Shop,Shoe Store


# Cluster 2

In [84]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Park,Playground,Tennis Court,Trail,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Dog Run
10,Downtown Toronto,1,Park,Playground,Trail,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
23,Central Toronto,1,Jewelry Store,Park,Sushi Restaurant,Trail,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store


# Cluster 3

In [85]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,2,Health Food Store,Trail,Pub,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store


# Cluster 4

In [86]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,3,Ice Cream Shop,Garden,Music Venue,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


# Cluster 5

In [87]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,4,Park,Swim School,Bus Line,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
